# Summary of Results
## Craniosynostosis Detection Group 
Pouria Mashour, Carson McLean, Chantal Shaib, Devin Singh, Seung-Eun Yi

**Note** - the plot images which follow each code block are embedded via Markdown using relative paths and therefore this `.ipynb` (and corresponding HTML version) must be kept within the project repository. Otherwise, the output images will be missing.

<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

// Rerun this cell to update Table of Contents
// https://github.com/kmahelona/ipython_notebook_goodies


<IPython.core.display.Javascript object>

# 2D GAN Results

In [ ]:
from GAN_2D import WGAN
import argparse
import numpy as np
import h5py
import scipy

def get_args():
    parser = argparse.ArgumentParser(description='Training 2DGAN:')
  
    parser.add_argument('--epochs', type=int, help='number of epochs for training')
    parser.add_argument('--batch_size', type=int, help='batch size for training')
    parser.add_argument('--save_int', type=int, help='interval to save data at')
    parser.add_argument('--out_file', type=str, help='file to save model (e.g. model.h5)')

    return parser.parse_args()

def pad_image(data):
        max_size = max(data[0].shape[0], data[0].shape[1])
        smaller_size = min(data[0].shape[0], data[0].shape[1])

        max_size = 224

        pad_size_l = (max_size - smaller_size)//2
        pad_size_r = smaller_size + (max_size - smaller_size)//2 

        print(pad_size_l)
        print(pad_size_r)
        uniform_data = np.zeros((data.shape[0], max_size, max_size, data.shape[-1]))
        uniform_data[:,pad_size_l:pad_size_r,pad_size_l:pad_size_r,:] = data

def load_data(num):
    """
    @param num: number indicating which angle of the 2D images you want
    """
    #dataset = h5py.File('/hpf/largeprojects/ccm/devin/plastics-data/general/create_2d_array/2D_data_color_4_ds8_uni_split_flips.h5'.format(num), 'r')
    dataset = h5py.File('/home/chantal/2D_data_grayscaled_4_ds8_uni.h5', 'r')
    data = dataset.get('data_im')
    labels = dataset.get('target')

    print(data.shape)
    #---------------- Pre-Process Data ----------------#
    syn_labels = [1,2,3,4,5,6]
    plag_labels = [7]
    norm_labels = [8]

    syn_data = []
    plag_data = []
    norm_data = []

    # separates data into classes
    for d,l in zip(data,labels):
        if l in syn_labels:
            syn_data += [d]
        elif l in norm_labels:
            norm_data += [d]
        elif l in plag_labels:
            plag_data += [d]

    # change this to train with synostosis/plagiocephaly/normal
    X_train = np.asarray(norm_data)
    batch = X_train.shape[0]
    channels = X_train.shape[3]
    height =X_train.shape[1]
    width = X_train.shape[2]
    X_train = np.reshape(X_train, (batch, channels, height, width))

    print("X_train shape", X_train.shape)

    return X_train


def train_model(data):
     wgan = WGAN()
     wgan.train(data, epochs=15000, batch_size=128, save=500)
     return    

def main():
    args = get_args()
    x_train = load_data(4)
    train_model(x_train)


if _name_ == '_main_':
    main()

In [ ]:
def sample_images(self, x_train, epoch):
   # change this to produce more examples
   rows, cols = 5, 5

   noise = np.random.normal(0, 1, (rows * cols, self.latent_dim))
   # uncomment second gen_imgs to visualize training set imgs
   gen_imgs = self.generator.predict(noise)
   #gen_imgs = x_train[:rows*cols, :, :, :]

   fig, axs = plt.subplots(rows, cols)
   c = 0
   for i in range(r):
       for j in range(c):
           axs[i,j].imshow(gen_imgs[c, :,:,0], cmap='gray')
           axs[i,j].axis('off')
           c += 1
   fig.savefig("gen/gen_wgan_%d" % epoch)
   plt.close()

def sample_losses(self, epoch, gloss, dloss):
   plt.figure(figsize=(10, 8\10))
   plt.plot(dloss, label='Discriminator loss')
   plt.plot(gloss, label='Generator loss')
   plt.xlabel('Epoch')
   plt.ylabel('Loss')
   plt.legend()
   plt.savefig('loss/loss_wgan_%d.png' % epoch)

# wasserstein loss function
def wasserstein_loss(self, y_true, y_pred):
   return K.mean(y_true*y_pred)

## Expected Output
![](notebook_images/real_2d_2.PNG)
## DCGAN
![](notebook_images/img_dcgan.png)
![](notebook_images/loss_dcgan.png)
## WGAN
![](notebook_images/gen_wgan_5000.PNG)
![](notebook_images/IMG_9881.PNG)

# 3D GAN Results

In [ ]:
from keras.layers.merge import _Merge
from keras.layers import Input, Dense, Flatten, Dropout, Add, Reshape
from keras.layers import BatchNormalization, Activation, MaxPooling1D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv3D, Deconv3D, Conv1D, UpSampling1D
from keras.models import Sequential, Model, load_model
from keras.optimizers import RMSprop, Adam
import keras.backend as K
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pylab as pl
import matplotlib.gridspec as gridspec
from tqdm import tqdm
import tensorflow as tf

# Plotting loss functions
def read_txt(input_path, wgan):
    dis_1 = []
    dis_2 = []
    gen = []
    with open(input_path, 'r') as f:
        for cnt, line in enumerate(f):
            nb = line.split(', ')
            dis_1.append(float(nb[1]))
            if wgan:
                gen.append(float(nb[2]))
            else:
                dis_2.append(float(nb[2]))
                gen.append(float(nb[3]))
    if wgan:
        return [dis_1], gen
    else:
        return [dis_1, dis_2], gen


def loss_plot(dis, gen, output_path, wgan):
    plt.figure()
    if wgan:
        plt.plot(dis[0], label="Critic_loss")
    else:
        plt.plot(dis[0], label="Discriminator_loss_real")
        plt.plot(dis[1], label="Discriminator_loss_fake")
    plt.plot(gen, label="Generator_loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend(loc= "upper right")

    plt.savefig(output_path)
    plt.show()
    plt.close()

## 3D DCGAN

In [ ]:
# 3D DCGAN

from dcgan_3d import GAN
import numpy as np
import h5py


def train_model(data_path, epochs, batch_size, save, out_file, input_shape, output_shape, train_class):
    print("Reading the data")

    data = h5py.File(data_path, 'r')
    x_data = data.get('data_im')[()]
    target = data.get('target')[()]
    l = x_data.shape[0]
    # Get the class we want to train
    index = []
    if train_class == 0:
        for i in range(l):
            if target[i][0] <= 6:
                index.append(i)
        x_data = x_data[index]
    if train_class == 1:
        for i in range(l):
            if target[i][0] == 7:
                index.append(i)
        x_data = x_data[index]
    if train_class == 2:
        for i in range(l):
            if target[i][0] == 8:
                index.append(i)
        x_data = x_data[index]
    
    
    model = GAN(learning_rate_gen=0.00002, learning_rate_disc=0.0002, in_shape=input_shape, out_shape=output_shape)
    model.train(x_data, epochs, save, batch_size, out_file)
    
    return model

# Train the model for class 0
dcgan = train_model(data_path, 2000, 64, 100, 'dcgan_3d.h5', (64, 64, 64, 1), (64, 64, 64, 1), 0)

# Plotting the losses:
dis_dc, gen_dc = read_txt('results_dcgan.txt')
loss_plot(dis_dc, gen_dc, 'losses_dcgan.png', wgan=0)

![](notebook_images/3ddcgan_loss.png)

In [ ]:
# Generating sample images : 3 images with 3 different angles (3x3)
rows = 3
noise = dcgan.generator_noise(rows)
generated_images = dcgan.generator.predict(noise)
u = generated_images[0].shape[0]

gs = gridspec.GridSpec(rows, 3)
plt.figure()
for i in range(rows):
    d = generated_images[i].reshape(u, u, u)
    ax = plt.subplot(gs[i, 0])
    plt.imshow(np.rot90(d[32,:,:], 2), cmap=plt.cm.get_cmap('gray_r', 20))
    plt.clim(0,1)
    ax = plt.subplot(gs[i, 1])
    plt.imshow(d[:,32,:], cmap=plt.cm.get_cmap('gray_r', 20))
    plt.clim(0,1)
    ax = plt.subplot(gs[i, 2])
    plt.imshow(np.rot90(d[:,:,32]), cmap=plt.cm.get_cmap('gray_r', 20))
    plt.colorbar()
    plt.clim(0,1)
plt.savefig("generated_images_dcgan.png")
plt.show()
plt.close()

![](notebook_images/3ddcgan_75.png)

## 3D WGAN

In [ ]:
# 3D WGAN

from wgan_3d import WGAN

# Train the model for class 0
data = h5py.File('/storage/general/data.h5', 'r')
x_data = data.get('data_im')[()]
target = data.get('target')[()]
l = x_data.shape[0]
index = []
    for i in range(l):
    if target[i][0] <= 6:
        index.append(i)
x_data = x_data[index]

wgan = WGAN()
wgan.train(x_data, epochs=40000, batch_size=128, sample_interval=100)


# Plotting the losses:
cri_wg, gen_wg = read_txt('results_wgan.txt')
loss_plot(cri_wg, gen_wg, 'losses_wgan.png', wgan=1)


![](notebook_images/3dwgan_loss.png)

In [ ]:
# Generating sample images : 3 images with 3 different angles (3x3)
rows = 3
noise = np.random.normal(0, 1, (3, wgan.latent_dim))
generated_images = wgan.generator.predict(noise)
u = generated_images[0].shape[0]

gs = gridspec.GridSpec(rows, 3)
plt.figure()
for i in range(rows):
    d = generated_images[i].reshape(u, u, u)
    ax = plt.subplot(gs[i, 0])
    plt.imshow(np.rot90(d[32,:,:], 2), cmap=plt.cm.get_cmap('gray_r', 20))
    plt.clim(0,1)
    ax = plt.subplot(gs[i, 1])
    plt.imshow(d[:,32,:], cmap=plt.cm.get_cmap('gray_r', 20))
    plt.clim(0,1)
    ax = plt.subplot(gs[i, 2])
    plt.imshow(np.rot90(d[:,:,32]), cmap=plt.cm.get_cmap('gray_r', 20))
    plt.colorbar()
    plt.clim(0,1)
plt.savefig("generated_images_wgan.png")
plt.show()
plt.close()

![](notebook_images/3dwgan_1500.png)

# 2D CNNs

## 2D CNN Baseline Results

In [ ]:
import os

from keras.layers import Input, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import roc_curve, auc
from keras.callbacks import TensorBoard
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras.utils import to_categorical
from keras.regularizers import l1
import tensorflow as tf

from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score, average_precision_score, recall_score, precision_score, log_loss
from sklearn.metrics import roc_auc_score, classification_report, roc_curve, confusion_matrix, precision_recall_curve
from sklearn.metrics import roc_curve, auc

import matplotlib
matplotlib.use('Agg')  # To disable X rendering so it works on terminal only
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import itertools

import math
import h5py
import numpy as np
from itertools import cycle
import argparse
import time
from scipy import interp
import warnings
# include this to avoid mem leaks issue with tf
K.clear_session()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.tensorflow_backend._get_available_gpus()
config = tf.ConfigProto( device_count = {'GPU': 1, 'CPU': 24})
sess = tf.Session(config=config)
keras.backend.set_session(sess)
# Get rid of ALL warning messages
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
K.tensorflow_backend._get_available_gpus()

#Paths for source data files
#Angle 1
filename='/home/dsingh/Documents/HPF_Files/Plastic_GANs/2D_CNN/data/2D_data_color_0_ds8_uni_split.h5'
path='/home/dsingh/Documents/HPF_Files/Plastic_GANs/2D_CNN/Models/2D_0_color_ds8'

#Angle 2
#filename='/home/dsingh/Documents/HPF_Files/Plastic_GANs/2D_CNN/data/2D_data_color_1_ds8_uni_split.h5'
#path='/home/dsingh/Documents/HPF_Files/Plastic_GANs/2D_CNN/Models/2D_1_color_ds8'

#Angle 3
#filename='/home/dsingh/Documents/HPF_Files/Plastic_GANs/2D_CNN/data/2D_data_color_2_ds8_uni_split.h5'
#path='/home/dsingh/Documents/HPF_Files/Plastic_GANs/2D_CNN/Models/2D_2_color_ds8'

#Angle 4
#filename='/home/dsingh/Documents/HPF_Files/Plastic_GANs/2D_CNN/data/2D_data_color_3_ds8_uni_split.h5'
#path='/home/dsingh/Documents/HPF_Files/Plastic_GANs/2D_CNN/Models/2D_3_color_ds8'

#Angle 5
#filename='/home/dsingh/Documents/HPF_Files/Plastic_GANs/2D_CNN/data/2D_data_color_4_ds8_uni_split.h5'
#path='/home/dsingh/Documents/HPF_Files/Plastic_GANs/2D_CNN/Models/2D_4_color_ds8'

#Calculating and Plotting AUROC for each of the predictions along with the micro-average AUROC
def test_model(model, testData, testTarget):
    y_predict = model.predict(testData)
    y_predict_non_category = [np.argmax(t) for t in y_predict]
    # print('The confusion matrix of the prediction is, ', confusion_matrix(y_test, y_predict_non_category))
    # print('The accuracy of the prediction is, ', accuracy_score(y_test, y_predict_non_category))

    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    lw = 2
    for i in range(3):
        fpr[i], tpr[i], _ = roc_curve(testTarget[:, i], y_predict0[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(testTarget.ravel(), y_predict0.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # Compute macro-average ROC curve and ROC area

    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(3)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(3):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= 3

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    plt.figure()
    plt.plot(fpr["micro"], tpr["micro"], label='Average ROC curve (area = {0:0.2f})'.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    # plt.plot(fpr["macro"], tpr["macro"],
    #        label='macro-average ROC curve (area = {0:0.2f})'
    #              ''.format(roc_auc["macro"]),
    #        color='navy', linestyle=':', linewidth=4)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    for i, color in zip(range(3), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='ROC curve of class {0} (area = {1:0.2f})'.format(i, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Model ' + str(m_i) + ': 2D ROC Baseline')
    plt.legend(loc="lower right")
    plt.savefig('2D_CNN_NoTL_AUC_' + str(time.time()) + '.png')


#Plotting the corresponding confusion matrix
def plot_confusion_matrix(cm, classes, normalize=True, title='Confusion matrix', cmap=plt.cm.Greens):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    Source: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    # Plot the confusion matrix
    plt.figure(figsize=(30, 30))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, size=20)
    plt.colorbar(aspect=3)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, size=14)
    plt.yticks(tick_marks, classes, size=14)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.

    # Labeling the plot
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), fontsize=16,
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.grid(None)
    plt.tight_layout()
    plt.ylabel('True label', size=16)
    plt.xlabel('Predicted label', size=16)
    plt.savefig(path+'/cf_baseline.png')

#Grouping outcome labels into 1 of 3 classes (0=Synostosis, 1=Plagiocephaly, 2=Normal)
def convert_to_categorical(target):
    target[target <= 6] = 0
    target[target == 7] = 1
    target[target == 8] = 2
    target = to_categorical(target)
    return target

#Loading the data
def load_data(filename):
    dataset = h5py.File(filename, 'r')
    trainData, trainTarget = dataset.get('train_data_im')[()], dataset.get('train_target')[()]
    validData, validTarget = dataset.get('valid_data_im')[()], dataset.get('valid_target')[()]
    testData, testTarget = dataset.get('test_data_im')[()], dataset.get('test_target')[()]
    trainTarget = convert_to_categorical(trainTarget)
    validTarget = convert_to_categorical(validTarget)
    testTarget = convert_to_categorical(testTarget)
    return trainData, trainTarget, validData, validTarget, testData, testTarget

trainData, trainTarget, validData, validTarget, testData, testTarget = load_data(filename)

#Creating baseline neural network model to be used without transfer learning
def create_model(input_shape, output_shape):

    model = Sequential()
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.8))

    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.6))

    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.6))

    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(Dropout(0.8))

    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(Dropout(0.8))

    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(Dropout(0.8))

    model.add(Flatten())
    model.add(Dense(8, activation='relu'))
    model.add(Dense(3, activation='softmax'))

    return model

trainTarget_i = [np.argmax(t) for t in trainTarget]
validTarget_i = [np.argmax(t) for t in validTarget]
testTarget_i = [np.argmax(t) for t in testTarget]

print("--- Data structures: --- ")
print("  Train Data:   " + str(trainData.shape))
print("  Train Target: " + str(trainTarget.shape))
print("  Valid Data:   " + str(validData.shape))
print("  Valid Target: " + str(validTarget.shape))
print("  Test Data:    " + str(testData.shape))
print("  Test Target:  " + str(testTarget.shape))
print("")

# dimensions of our images.
img_width = trainData.shape[2]
img_height = trainData.shape[1]

nb_train_samples = trainData.shape[0]
nb_validation_samples = validData.shape[0]
#epochs = 1  # need to change these values
#batch_size = 1  # need to change these values

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

output_shape = len(trainTarget[0])

print("--- Size of input & output: --- ")
print("  Input Shape:  " + str(input_shape))
print("  Output Shape: " + str(output_shape))
print("")

model = create_model(input_shape, output_shape)

#opt = keras.optimizers.SGD(lr=0.00001)
opt = keras.optimizers.Adam(lr=0.00004)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

class_weights = class_weight.compute_class_weight('balanced', np.unique(trainTarget_i), trainTarget_i)
early_stopping_monitor = EarlyStopping(patience=10)

history=model.fit(
    x=trainData,
    y=trainTarget,
    batch_size=50,
    epochs=500,
    validation_data=(validData, validTarget),
    class_weight=class_weights,
    callbacks=[early_stopping_monitor],
    verbose=1
)

#Save the model
model.save_weights(path+'/2D_CNN_colour_ds8_baseline.h5')
print (model.summary())

#Evaluate the Model
results = model.evaluate(testData, testTarget)
predictions = model.predict_classes(testData)
print(model.metrics_names)
print(results)

# Create Accuracy and Loss Over Time
history_dict = history.history
history_dict.keys()

# dict_keys(['loss', 'acc', 'val_acc', 'val_loss'])
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.figure(figsize = (10,8))
plt.style.use('fivethirtyeight')
# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig(path+'/NNloss_baseline.png')

plt.clf()   # clear figure
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

plt.figure(figsize=(10, 8))
plt.style.use('fivethirtyeight')
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig(path+'/NNAccuracy_baseline.png')

#calculations for confusion matrix
y_hat_train = model.predict_classes(trainData, verbose=1)
confusion_matrix_train = confusion_matrix(trainTarget_i, y_hat_train)
print("Data Distribution for Training Set: ")
tr_unique, tr_counts = np.unique(trainTarget_i, return_counts=True)
print(dict(zip(tr_unique, tr_counts)))
print("Confusion Matrix for Training Set: ")
print(confusion_matrix_train)
print("")

y_hat_valid = model.predict_classes(validData, verbose=1)
confusion_matrix_valid = confusion_matrix(validTarget_i, y_hat_valid)
print("Data Distribution for Validation Set: ")
va_unique, va_counts = np.unique(validTarget_i, return_counts=True)
print(dict(zip(va_unique, va_counts)))
print("Confusion Matrix for Validation Set: ")
print(confusion_matrix_valid)
print("")

y_hat_test = model.predict_classes(testData, verbose=1)
cm = confusion_matrix(testTarget_i, y_hat_test)
print("Data Distribution for Test Set: ")
te_unique, te_counts = np.unique(testTarget_i, return_counts=True)
print(dict(zip(te_unique, te_counts)))
print("Confusion Matrix for Test Set: ")
print(cm)
print("")
plot_confusion_matrix(cm, classes=['Synostosis', 'Plagiocephaly', 'Normal'], title='Confusion Matrix')

test_model(model, testData, testTarget)

![](2D_CNN_Final/2D_CNN_Baseline/Baseline_2D_CNN_1.png)
![](2D_CNN_Final/2D_CNN_Baseline/Baseline_2D_CNN_2.png)
![](2D_CNN_Final/2D_CNN_Baseline/Baseline_2D_CNN_3.png)
![](2D_CNN_Final/2D_CNN_Baseline/Baseline_2D_CNN_4.png)
![](2D_CNN_Final/2D_CNN_Baseline/Baseline_2D_CNN_5.png)

## 2D Ensemble Results

In [ ]:
import pandas as pd
import numpy as np

import itertools
from itertools import cycle
import h5py
import time
import sys
import argparse
import math
import os


# Plotting tools
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.metrics import accuracy_score, average_precision_score, recall_score, precision_score, log_loss, roc_auc_score, classification_report, roc_curve, confusion_matrix, precision_recall_curve, auc
from sklearn.utils import shuffle, class_weight
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils.fixes import signature
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import binarize
from scipy import stats
from scipy import interp


# Get rid of ALL warning messages
import os
import warnings
import tensorflow as tf
import keras
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from keras import backend as K
K.tensorflow_backend._get_available_gpus()
from keras.applications import InceptionResNetV2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import TensorBoard
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform
from keras.layers import Conv3D, MaxPool3D
from keras.layers import Input, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.models import Model


# include this to avoid mem leaks issue with tf
K.clear_session()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
config = tf.ConfigProto( device_count = {'GPU': 1, 'CPU': 24})
sess = tf.Session(config=config)
keras.backend.set_session(sess)





#loading models
path='/home/pouriamashouri/project'  # Where the individual graphs will be saved

#Paths for source data files
#Angle 1
filename1 = "/home/pouriamashouri/project/data/2D_data_color_0_ds8_uni_split_flips.h5"
model_name1 = "/home/pouriamashouri/project/Models/2D_CNN_colour_0_ds8.h5"

#Angle 2
filename2 = "/home/pouriamashouri/project/data/2D_data_color_1_ds8_uni_split_flips.h5"
model_name2 = "/home/pouriamashouri/project/Models/2D_CNN_colour_1_ds8.h5"

#Angle 3
filename3 = "/home/pouriamashouri/project/data/2D_data_color_2_ds8_uni_split_flips.h5"
model_name3 = "/home/pouriamashouri/project/Models/2D_CNN_colour_2_ds8.h5"

#Angle 4
filename4 = "/home/pouriamashouri/project/data/2D_data_color_3_ds8_uni_split_flips.h5"
model_name4 = "/home/pouriamashouri/project/Models/2D_CNN_colour_3_ds8.h5"

#Angle 5
filename5 = "/home/pouriamashouri/project/data/2D_data_color_4_ds8_uni_split_flips.h5"
model_name5 = "/home/pouriamashouri/project/Models/2D_CNN_colour_4_ds8.h5"


# Plotting the AUROCs for each of the individual models

def test_model(testData, testTarget, m_i, model=None, y_predict=None):
    if (y_predict is None):
        y_predict = model.predict(testData)
        
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    lw = 2
    for i in range(3):
        fpr[i], tpr[i], _ = roc_curve(testTarget[:, i], y_predict[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(testTarget.ravel(), y_predict.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # Compute macro-average ROC curve and ROC area

    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(3)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(3):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= 3

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    plt.figure()
    plt.plot(fpr["micro"], tpr["micro"], label='Average ROC curve (area = {0:0.2f})'.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    for i, color in zip(range(3), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw, label='ROC curve of class {0} (area = {1:0.2f})'.format(i, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Model ' + str(m_i) + ': 2D ROC Baseline')
    plt.legend(loc="lower right")

    plt.savefig('2D_CNN_Ensemble_AUROC.png')


def plot_confusion_matrix(cm, classes, normalize=True, title='Model 0 Confusion matrix', cmap=plt.cm.Greens, m_i=0):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    Source: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    # Plot the confusion matrix
    plt.figure(figsize=(20, 14))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, size=24)
    plt.colorbar(aspect=4)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, size=14)
    plt.yticks(tick_marks, classes, size=14)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.

    # Labeling the plot
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), fontsize=20,
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.grid(None)
    plt.tight_layout()
    plt.ylabel('True label', size=18)
    plt.xlabel('Predicted label', size=18)
    plt.savefig('2D_CNN_Ensemble_CM.png')


#Loading and processing the data for each model
def convert_to_categorical(target):
    target[target <= 6] = 0
    target[target == 7] = 1
    target[target == 8] = 2
    target = to_categorical(target)
    return target


def load_data(filename):
    dataset = h5py.File(filename, 'r')
    trainData, trainTarget = dataset.get('train_data_im')[()], dataset.get('train_target')[()]
    validData, validTarget = dataset.get('valid_data_im')[()], dataset.get('valid_target')[()]
    testData, testTarget = dataset.get('test_data_im')[()], dataset.get('test_target')[()]
    trainTarget = convert_to_categorical(trainTarget)
    validTarget = convert_to_categorical(validTarget)
    testTarget = convert_to_categorical(testTarget)
    return trainData, trainTarget, validData, validTarget, testData, testTarget




def make_prediction(filename, model_name, m_i):
    trainData, trainTarget, validData, validTarget, testData, testTarget = load_data(filename)

    testTarget_i = [np.argmax(t) for t in testTarget]

    # Defining the model input and output shape
    img_width = testData.shape[2]
    img_height = testData.shape[1]

    if K.image_data_format() == 'channels_first':
        input_shape = (3, img_width, img_height)
    else:
        input_shape = (img_width, img_height, 3)
    output_shape = len(testTarget[0])

    #Defining each of the model CNNs, loading each model, and then compiling the model after the load
    conv_base = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

    #Model
    model = Sequential()
    model.add(conv_base)
    model.add(Flatten())
    model.add(Dense(8, activation='relu'))
    model.add(Dense(3, activation='softmax'))


    with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        model.load_weights(model_name)

    opt = keras.optimizers.Adam(lr=0.00004)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    #Make Predictions
    y_hat_test_all = model.predict(testData, verbose=1)
    return y_hat_test_all, np.array(testTarget_i)



angle1_prediction, angle1_target = make_prediction(filename1, model_name1, 0)
angle2_prediction, angle2_target = make_prediction(filename2, model_name2, 1)
angle3_prediction, angle3_target = make_prediction(filename3, model_name3, 2)
angle4_prediction, angle4_target = make_prediction(filename4, model_name4, 3)
angle5_prediction, angle5_target = make_prediction(filename5, model_name5, 4)


if ((angle1_target==angle2_target).all() and (angle1_target==angle3_target).all() and (angle1_target==angle4_target).all() and (angle1_target==angle5_target).all()):
    print("All targets are the same")
else:
    print("Error, targets dont match. Exiting")
    sys.exit(1)

target = angle1_target


final_prediction = (angle1_prediction + angle2_prediction + angle3_prediction + angle4_prediction + angle5_prediction)/5
final_prediction = final_prediction.reshape(-1,3)
final_prediction_c = np.argmax(final_prediction, axis=1).reshape(-1,1)

confusion_matrix_ensemble = confusion_matrix(target, final_prediction_c)
plot_confusion_matrix(confusion_matrix_ensemble, classes=['Synostosis', 'Plagiocephaly', 'Other'], title='Ensemble Confusion Matrix', m_i="ensemble")
target = to_categorical(target)
test_model(None, target, m_i="ensemble", model=None, y_predict=final_prediction)

![](2D_CNN_Final/2D_CNN_Ensemble/2D_CNN_Ensemble_AUROC.png)
![](2D_CNN_Final/2D_CNN_Ensemble/2D_CNN_Ensemble_CM.png)

## 2D Transfer Learning CNN Results

In [ ]:
import os
# Get rid of ALL warning messages
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from keras.applications.inception_v3 import preprocess_input
from keras.applications import InceptionResNetV2

from keras import backend as K
K.tensorflow_backend._get_available_gpus()
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.layers import Dropout, Input, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import roc_curve, auc
from keras.callbacks import TensorBoard
from keras.regularizers import l1
import tensorflow as tf

# Plotting tools
import matplotlib
matplotlib.use('Agg')  # To disable X rendering so it works on terminal only
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import itertools
from itertools import cycle

from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score, average_precision_score, recall_score, precision_score, log_loss
from sklearn.metrics import roc_auc_score, classification_report, roc_curve, confusion_matrix, precision_recall_curve
from sklearn.metrics import roc_curve, auc
from scipy import interp

import h5py
import numpy as np
from itertools import cycle
import argparse
import time
import math
from scipy import interp

# include this to avoid mem leaks issue with tf
K.clear_session()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
config = tf.ConfigProto( device_count = {'GPU': 1, 'CPU': 24})
sess = tf.Session(config=config)
keras.backend.set_session(sess)


#Paths for source data files
#Angle 1
filename='/home/dsingh/Documents/HPF_Files/Plastic_GANs/2D_CNN/data/2D_data_color_0_ds8_uni_split.h5'
path='/home/dsingh/Documents/HPF_Files/Plastic_GANs/2D_CNN/Models/2D_0_color_ds8'

#Angle 2
#filename='/home/dsingh/Documents/HPF_Files/Plastic_GANs/2D_CNN/data/2D_data_color_1_ds8_uni_split.h5'
#path='/home/dsingh/Documents/HPF_Files/Plastic_GANs/2D_CNN/Models/2D_1_color_ds8'

#Angle 3
#filename='/home/dsingh/Documents/HPF_Files/Plastic_GANs/2D_CNN/data/2D_data_color_2_ds8_uni_split.h5'
#path='/home/dsingh/Documents/HPF_Files/Plastic_GANs/2D_CNN/Models/2D_2_color_ds8'

#Angle 4
#filename='/home/dsingh/Documents/HPF_Files/Plastic_GANs/2D_CNN/data/2D_data_color_3_ds8_uni_split.h5'
#path='/home/dsingh/Documents/HPF_Files/Plastic_GANs/2D_CNN/Models/2D_3_color_ds8'

#Angle 5
#filename='/home/dsingh/Documents/HPF_Files/Plastic_GANs/2D_CNN/data/2D_data_color_4_ds8_uni_split.h5'
#path='/home/dsingh/Documents/HPF_Files/Plastic_GANs/2D_CNN/Models/2D_4_color_ds8'

#Calculating and Plotting AUROC for each of the predictions along with the micro-average AUROC
def test_model(model, testData, testTarget):
    y_predict = model.predict(testData)
    y_predict_non_category = [np.argmax(t) for t in y_predict]
    # print('The confusion matrix of the prediction is, ', confusion_matrix(y_test, y_predict_non_category))
    # print('The accuracy of the prediction is, ', accuracy_score(y_test, y_predict_non_category))

    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    lw = 2
    for i in range(3):
        fpr[i], tpr[i], _ = roc_curve(testTarget[:, i], y_predict[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(testTarget.ravel(), y_predict.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # Compute macro-average ROC curve and ROC area

    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(3)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(3):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= 3

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    plt.figure()
    plt.plot(fpr["micro"], tpr["micro"], label='micro-average ROC curve (area = {0:0.2f})'.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
             label='macro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["macro"]),
             color='navy', linestyle=':', linewidth=4)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    for i, color in zip(range(3), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw, label='ROC curve of class {0} (area = {1:0.2f})'.format(i, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC for 2D Baseline')
    plt.legend(loc="lower right")

    plt.savefig('Baseline_2D_AUC_' + str(time.time()) + '.png')

#Plotting the corresponding confusion matrix
def plot_confusion_matrix(cm, classes, normalize=True, title='Confusion matrix', cmap=plt.cm.Greens):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    Source: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    # Plot the confusion matrix
    plt.figure(figsize=(20, 14))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, size=24)
    plt.colorbar(aspect=4)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, size=14)
    plt.yticks(tick_marks, classes, size=14)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.

    # Labeling the plot
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), fontsize=20,
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.grid(None)
    plt.tight_layout()
    plt.ylabel('True label', size=18)
    plt.xlabel('Predicted label', size=18)
    plt.savefig(path+'/cf.png')

#Grouping outcome labels into 1 of 3 classes (0=Synostosis, 1=Plagiocephaly, 2=Normal)
def convert_to_categorical(target):
    target[target <= 6] = 0
    target[target == 7] = 1
    target[target == 8] = 2
    target = to_categorical(target)
    return target

#Loading the data
def load_data(filename):
    dataset = h5py.File(filename, 'r')
    trainData, trainTarget = dataset.get('train_data_im')[()], dataset.get('train_target')[()]
    validData, validTarget = dataset.get('valid_data_im')[()], dataset.get('valid_target')[()]
    testData, testTarget = dataset.get('test_data_im')[()], dataset.get('test_target')[()]
    trainTarget = convert_to_categorical(trainTarget)
    validTarget = convert_to_categorical(validTarget)
    testTarget = convert_to_categorical(testTarget)
    return trainData, trainTarget, validData, validTarget, testData, testTarget

trainData, trainTarget, validData, validTarget, testData, testTarget = load_data(filename)

#Creating baseline neural network model to be used without transfer learning
def create_model(input_shape, output_shape):
    conv_base = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    #conv_base=applications.InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

    model = Sequential()
    model.add(conv_base)
    #model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    #model.add(MaxPooling2D((2, 2)))
    #model.add(Dropout(0.8))

    #model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.6))

    #model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.6))

    #model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    #model.add(Dropout(0.8))

    #model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    #model.add(Dropout(0.8))

    model.add(Flatten())
    model.add(Dense(8, activation='relu'))
    model.add(Dense(3, activation='softmax'))

    return model

trainTarget_i = [np.argmax(t) for t in trainTarget]
validTarget_i = [np.argmax(t) for t in validTarget]
testTarget_i = [np.argmax(t) for t in testTarget]

print("--- Data structures: --- ")
print("  Train Data:   " + str(trainData.shape))
print("  Train Target: " + str(trainTarget.shape))
print("  Valid Data:   " + str(validData.shape))
print("  Valid Target: " + str(validTarget.shape))
print("  Test Data:    " + str(testData.shape))
print("  Test Target:  " + str(testTarget.shape))
print("")

# dimensions of our images.
img_width = trainData.shape[2]
img_height = trainData.shape[1]

nb_train_samples = trainData.shape[0]
nb_validation_samples = validData.shape[0]
#epochs = 1  # need to change these values
#batch_size = 1  # need to change these values

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

output_shape = len(trainTarget[0])

print("--- Size of input & output: --- ")
print("  Input Shape:  " + str(input_shape))
print("  Output Shape: " + str(output_shape))
print("")

model = create_model(input_shape, output_shape)

#opt = keras.optimizers.SGD(lr=0.00001)
opt = keras.optimizers.Adam(lr=0.00004)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

class_weights = class_weight.compute_class_weight('balanced', np.unique(trainTarget_i), trainTarget_i)
early_stopping_monitor = EarlyStopping(patience=10)

history=model.fit(
    x=trainData,
    y=trainTarget,
    batch_size=50,
    epochs=500,
    validation_data=(validData, validTarget),
    class_weight=class_weights,
    callbacks=[early_stopping_monitor],
    verbose=1
)

#Save the model
model.save_weights(path+'/2D_CNN_colour_ds8.h5')

#Evaluate the Model
results = model.evaluate(testData, testTarget)
predictions = model.predict_classes(testData)
print(model.metrics_names)
print(results)

# Create Accuracy and Loss Over Time
history_dict = history.history
history_dict.keys()

# dict_keys(['loss', 'acc', 'val_acc', 'val_loss'])
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.figure(figsize = (10,8))
plt.style.use('fivethirtyeight')
# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig(path+'/NNloss.png')

plt.clf()   # clear figure
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

plt.figure(figsize=(10, 8))
plt.style.use('fivethirtyeight')
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig(path+'/NNAccuracy.png')

probs = model.predict_proba(testData) #[:, 1]

#calculations for confusion matrix
y_hat_train = model.predict_classes(trainData, verbose=1)
confusion_matrix_train = confusion_matrix(trainTarget_i, y_hat_train)
print("Data Distribution for Training Set: ")
tr_unique, tr_counts = np.unique(trainTarget_i, return_counts=True)
print(dict(zip(tr_unique, tr_counts)))
print("Confusion Matrix for Training Set: ")
print(confusion_matrix_train)
print("")

y_hat_valid = model.predict_classes(validData, verbose=1)
confusion_matrix_valid = confusion_matrix(validTarget_i, y_hat_valid)
print("Data Distribution for Validation Set: ")
va_unique, va_counts = np.unique(validTarget_i, return_counts=True)
print(dict(zip(va_unique, va_counts)))
print("Confusion Matrix for Validation Set: ")
print(confusion_matrix_valid)
print("")

y_hat_test = model.predict_classes(testData, verbose=1)
cm = confusion_matrix(testTarget_i, y_hat_test)
print("Data Distribution for Test Set: ")
te_unique, te_counts = np.unique(testTarget_i, return_counts=True)
print(dict(zip(te_unique, te_counts)))
print("Confusion Matrix for Test Set: ")
print(cm)
print("")
plot_confusion_matrix(cm, classes=['Synostosis', 'Plagiocephaly', 'Other'], title='Confusion Matrix')

test_model(model, testData, testTarget)

![](2D_CNN_Final/2D_CNN_TransferLearning/2D_CNN_TL_1.png)
![](2D_CNN_Final/2D_CNN_TransferLearning/2D_CNN_TL_1_CM.png)
![](2D_CNN_Final/2D_CNN_TransferLearning/2D_CNN_TL_2.png)
![](2D_CNN_Final/2D_CNN_TransferLearning/2D_CNN_TL_2_CM.png)
![](2D_CNN_Final/2D_CNN_TransferLearning/2D_CNN_TL_3.png)
![](2D_CNN_Final/2D_CNN_TransferLearning/2D_CNN_TL_3_CM.png)
![](2D_CNN_Final/2D_CNN_TransferLearning/2D_CNN_TL_4.png)
![](2D_CNN_Final/2D_CNN_TransferLearning/2D_CNN_TL_4_CM.png)
![](2D_CNN_Final/2D_CNN_TransferLearning/2D_CNN__TL_5.png)
![](2D_CNN_Final/2D_CNN_TransferLearning/2D_CNN_TL_5_CM.png)


# 3D CNNs

## 3D ResNet-18 Results

In [ ]:
# Restrict to one GPU in multi-GPU systems
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from collections import Counter
import h5py
from itertools import cycle, product
from keras import applications
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
from resnet3d import Resnet3DBuilder # https://github.com/JihongJu/keras-resnet3d
from scipy import interp
from shutil import copy
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
import time


def plot_confusion_matrix(cm, classes, normalize=True, title='Confusion matrix', cmap=plt.cm.Greens):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    Source: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    # Plot the confusion matrix
    plt.figure(figsize=(20, 14))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, size=24)
    plt.colorbar(aspect=4)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, size=14)
    plt.yticks(tick_marks, classes, size=14)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.

    # Labeling the plot
    for i, j in product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), fontsize=20,
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.grid(None)
    plt.tight_layout()
    plt.ylabel('True label', size=18)
    plt.xlabel('Predicted label', size=18)
    plt.savefig(output_dir + '/3D_ResNet_Confusion_Matrix.png')

def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
    axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.savefig(output_dir + '/3D_ResNet_Acc_Loss.png')

def create_train_test(datapath, test_size=0.3):
    with h5py.File(datapath, 'r') as dataset:
        data = dataset.get('data_im')[()]
        target = dataset.get('target')[()]
        for i in range(len(target)):
            if target[i][0] <= 6:
                target[i][0] = 0
            if target[i][0] == 7:
                target[i][0] = 1
            if target[i][0] == 8:
                target[i][0] = 2
        X_train, X_test, y_train, y_test = train_test_split(data, target,
                                                            test_size=test_size)
        X_train = X_train.reshape(X_train.shape[0], X_train.shape[1],
                                  X_train.shape[2], X_train.shape[3], 1)
        X_test = X_test.reshape(X_test.shape[0], X_test.shape[1],
                                X_test.shape[2], X_test.shape[3], 1)
        y_train = to_categorical(y_train, 3)
        y_test = to_categorical(y_test, 3)
        return X_train, X_test, y_train, y_test

# Setup the output directory and copy this same script to the directory for future reference
output_dir = "../output/" + str(time.strftime("%m-%d-%H:%M"))
os.mkdir(output_dir)
copy(__file__, output_dir)

# Download the data from: /hpf/largeprojects/ccm/devin/plastics-data/???
datapath = "../data/data.h5"
test_size = 0.20
X_train, X_test, Y_train, Y_test = create_train_test(datapath, test_size)


print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

model = Resnet3DBuilder.build_resnet_18((128, 128, 128, 1), 3)
adam = Adam(lr=0.000003)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
earlystop = EarlyStopping(monitor = 'val_acc',
                          min_delta = 0.01,
                          patience = 3,
                          verbose = 2,
                          restore_best_weights = True)
callback_list = [earlystop]

model_info = model.fit(X_train, Y_train,
                       epochs = 25,
                       batch_size = 8,
                       validation_split = 0.10,
                       callbacks = callback_list)

preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

plot_model_history(model_info)

# model.summary()

Y_hat_test = model.predict(X_test, verbose=1)
cm = confusion_matrix(Y_test.argmax(axis=1), Y_hat_test.argmax(axis=1))
print("Data Distribution for Test Set: ")
te_unique, te_counts = np.unique(Y_test, return_counts=True)
print(dict(zip(te_unique, te_counts)))
print("Confusion Matrix for Test Set: ")
print(cm)
print("")
plot_confusion_matrix(cm, classes=['<6', '7', '8'], title='Confusion Matrix', normalize=True)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
lw = 2
for i in range(3):
    fpr[i], tpr[i], _ = roc_curve(Y_test[:, i], Y_hat_test[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(Y_test.ravel(), Y_hat_test.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area

all_fpr = np.unique(np.concatenate([fpr[i] for i in range(3)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(3):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= 3

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["micro"], tpr["micro"], label='micro-average ROC curve (area = {0:0.2f})'.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(3), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
                   ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.05])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC for 3D ResNet')
plt.legend(loc="lower right")

plt.savefig(output_dir + '/3D_ResNet_AUC.png')


model.save(output_dir + '/model.h5')


![](notebook_images/3D_ResNet_AUROC.png)
![](notebook_images/3D_ResNet_CM.png)

## 3D ResNet-50 with Transfer Learning Results

In [ ]:
# Restrict to one GPU in multi-GPU systems
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from collections import Counter
import h5py
from keras import applications
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
from resnet3d import Resnet3DBuilder, basic_block # https://github.com/JihongJu/keras-resnet3d
from shutil import copy
from sklearn.model_selection import train_test_split
import time
from presentation_plots import plot_confusion_matrix, plot_rocauc, plot_model_history


def create_train_test(datapath, test_size=0.3):
    with h5py.File(datapath, 'r') as dataset:
        data = dataset.get('data_im')[()]
        target = dataset.get('target')[()]
        print(data.shape)
        print(target.shape)

        for i in range(len(target)):
            if target[i][0] <= 6:
                target[i][0] = 0
            if target[i][0] == 7:
                target[i][0] = 1
            if target[i][0] == 8:
                target[i][0] = 2
        X_train, X_test, y_train, y_test = train_test_split(data, target,
                                                            test_size=test_size)
        X_train = X_train.reshape(X_train.shape[0], X_train.shape[1],
                                  X_train.shape[2], X_train.shape[3], 1)
        X_test = X_test.reshape(X_test.shape[0], X_test.shape[1],
                                X_test.shape[2], X_test.shape[3], 1)
        y_train = to_categorical(y_train, 3)
        y_test = to_categorical(y_test, 3)
        return X_train, X_test, y_train, y_test

# Setup the output directory and copy this same script to the directory for future reference
output_dir = "output/" + str(time.strftime("%m-%d-%H:%M"))
os.mkdir(output_dir)
copy(__file__, output_dir)

# Download the data from: /hpf/largeprojects/ccm/devin/plastics-data/???
datapath = "data/data.h5"
test_size = 0.20
X_train, X_test, Y_train, Y_test = create_train_test(datapath, test_size)


print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

model = Resnet3DBuilder.build_resnet_50((128, 128, 128, 1), 3)
# model = Resnet3DBuilder.build((128, 128, 128, 1), 3, basic_block, [1, 1, 1, 1], reg_factor=1e-4)

# Uncomment the following block to do transfer learning!
model.layers[-1].name = "dense_resnet_1" # Rename final dense layer so correct number of output classes is used (3 instead of MNIST 10)
# MNIST build_resnet_18
# model.load_weights('/home/carsonmclean/dev/csc2541/csc2541/output/04-03-03:37/model.h5',
#                    by_name = True)
# MNIST [1,1,1,1]
# model.load_weights('/home/carsonmclean/dev/csc2541/csc2541/output/04-09-00:25/model.h5',
#                    by_name = True)

# ModelNet40/PointNet build_resnet_18 dim = 64
# model.load_weights('/home/carsonmclean/dev/csc2541/csc2541/output/04-09-17:40/model.h5',
#                    by_name = True)
# ModelNet40/PointNet build_resnet_18 dim = 128
# model.load_weights('/home/carsonmclean/dev/csc2541/csc2541/output/04-10-08:41/model.h5',
#                    by_name = True)
# ModelNet40/PointNet build_resnet_50 dim = 64
model.load_weights('/home/carsonmclean/dev/csc2541/csc2541/output/04-10-08:54/model.h5',
                   by_name = True)

# End of transfer learning

adam = Adam(lr=0.000001)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
earlystop = EarlyStopping(monitor = 'val_acc',
                          min_delta = 0.01,
                          patience = 6,
                          verbose = 2,
                          restore_best_weights = True)
callback_list = [earlystop]

model_info = model.fit(X_train, Y_train,
                       epochs = 50,
                       batch_size = 4,
                       validation_split = 0.10,
                       callbacks = callback_list)

preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

Y_hat_test = model.predict(X_test, verbose=1)

plot_confusion_matrix(Y_test,
                      Y_hat_test,
                      ['Synostosis', 'Plagiocephaly', 'Normal'],
                      "3D ResNet",
                      output_dir)
plot_model_history(model_info, "3D ResNet", output_dir)
plot_rocauc(Y_test, Y_hat_test, "3D ResNet", output_dir)


model.save(output_dir + '/model.h5')


![](notebook_images/3D_ResNet-50_TL_AUC.png)
![](notebook_images/3D_ResNet-50_TL_Confusion_Matrix.png)

<IPython.core.display.Javascript object>